In [1]:
import pandas as pd

In [20]:
#Archivo DGIL
dgil=pd.read_excel('C:/Users/claudio.pacheco/Desktop/Lista de precios de medicamentos_Diciembre_2022.xlsx',skiprows=(4), usecols=(range(5)))
dgil = dgil.iloc[:-2]
dgil.columns=dgil.columns.str.lower()
#Renmbrar penúltima y última columna
fourth_column_name = dgil.columns[3]
fifth_column_name = dgil.columns[4]
dgil.rename(columns={fourth_column_name:'pres',fifth_column_name:'precio'}, inplace=True)
#Eliminar filas vacías
dgil=dgil.dropna()
#Elimi
dgil

,empresa,producto,principio activo,pres,precio
0,"Abbvie Famacéuticos, S.A. de C.V.",VENCLEXTA,Venetoclax,Caja con un frasco con 120 tabletas de 100 mg.,149594.00
1,"Abbvie Famacéuticos, S.A. de C.V.",VENCLEXTA,Venetoclax,Tratamiento de inicio. Caja con 4 carteras sem...,32287.00
2,"Abbvie Famacéuticos, S.A. de C.V.",MAVYRET,Glecaprevir / Pribrenstavir,Caja con 4 cajas individuales con 7 blisters c...,96912.00
3,"Abbvie Famacéuticos, S.A. de C.V.",RIM-VOQ,Upadacitinib,Caja de cartón con 28 tabletas de 15 mg en env...,22612.88
4,"Abbvie Famacéuticos, S.A. de C.V.",SKYRIZI,Risankizumab,Caja de cartón con 2 jeringas prellenadas 0.83...,90600.00
...,...,...,...,...,...
375,"Takeda México, S.A. de C.V.",TURAZIVE,Febuxostat,Frasco con 30 tabletas 80 mg,917.00
377,"UCB de México, S.A de C.V",BRIVIACT,Brivaracetam,Caja con 28 tabletas de 25 mg,2060.00
378,"UCB de México, S.A de C.V",BRIVIACT,Brivaracetam,Caja con 28 tabletas de 50 mg,2060.00
379,"UCB de México, S.A de C.V",BRIVIACT,Brivaracetam,Caja con 28 tabletas de 100 mg,2060.00
